# Regularized Linear Models 규제 선형 회귀
다항식이 복잡해져서 회귀계수가 매우 크게 설정이되면서 과대적합이 되고 평가데이터세트에 대해서 형편없는 예측 성능을 보이게 된다.


비용함수의 최소값을 구하는 것이 모델이 추구하는 목적이므로, 비용함수의 최소값을 구하는데 𝝰 제약을 걸어 과적합을 방지하는 것을 규제라고 한다.


- Lasso L1방식의 규제 적용
- Ridge L2방식의 규제 적용
- ElasticNet L1, L2규제를 결합한 모델. 특성이 많은 데이터셋에 적용. L1규제로 특성개수를 줄이고, L2규제로 계수값의 크기도 조정할 수 있다.


**비용함수 목표**


$
\text{비용함수 목표} = \min \left(\text{RSS}(w) + \alpha \times W\right)
$


이 수식은 규제를 적용한 비용함수를 의미하며, 다음과 같은 요소들로 이루어져 있다:


1. **RSS(w)**: 이 부분은 Residual Sum of Squares의 약자로, 잔차 제곱합을 의미한다. 선형 회귀 모델에서 주로 사용하는 손실 함수로, 각 데이터 포인트에서의 예측 값과 실제 값 간의 차이를 제곱한 것들의 합이다. 즉, 모델의 예측 오차를 측정하는 부분이다. 수식으로는 다음과 같이 표현된다:
   $
   \text{RSS}(w) = \sum_{i=1}^n \left(y_i - \hat{y}_i\right)^2
   $
  여기서 $y_i$는 실제 값, $\hat{y}_i$는 모델의 예측 값이다.


2. **$\alpha$**: 규제 강도를 나타내는 하이퍼파라미터이다. 이 값이 클수록 규제의 효과가 커지고, 작을수록 규제의 효과가 줄어든다. 모델이 과적합되기 쉬운 경우, $\alpha$를 크게 설정하여 가중치를 제어할 수 있다.


3. **$W$**: 가중치들의 규제 항을 의미한다. 이는 가중치의 크기에 페널티를 부여하는 부분으로, 모델의 복잡도를 조절하는 역할을 한다.


  - L1 규제: $W = \sum_{j=1}^p |w_j|$
  - L2 규제: $W = \sum_{j=1}^p w_j^2$


수식을 다시 설명하면, 비용함수의 목표는 잔차 제곱합(RSS)과 규제 항($\alpha \times W$)을 더한 값을 최소화하는 것이다. 즉, 이 비용함수의 최적화 목표는 두 가지를 달성하고자 한다:


1. 모델의 예측 오차(RSS)를 줄이는 것.
2. 모델의 복잡도를 줄여서 가중치의 크기를 제어하는 것($\alpha \times W$).


**적합합 규제를 선택하려면 :**
1. **Lasso (L1 규제)**
 - 불필요한 피처를 자동으로 제거할 때 유용하다.
 - 많은 피처 중 일부만 중요할 때 사용하면 스파스한 모델을 만든다.
2. **Ridge (L2 규제)**
 - 모든 피처가 유의미하고 예측에 기여한다고 생각될 때 적합하다.
 - 피처가 많고 과적합을 방지하고 싶을 때 사용한다.
3. **Elastic Net (L1 + L2 규제)**
 - 피처를 일부 제거하면서도 나머지의 가중치도 줄이고 싶을 때 유용하다.
 - 상관관계가 높은 피처가 있을 때 선택을 안정적으로 한다.



## L2
- L2 방식의 규제를 구현한 Ridge 클래스를 사용할 수 있다.
- 모든 피처의 회귀계수를 규제해 과적합을 방지한다.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error

In [4]:
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing()

california_housing_df = pd.DataFrame(california_housing.data, columns = california_housing.feature_names)

california_housing_df['MedHouseVal'] = california_housing.target

california_housing_df.head()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [5]:
# 데이터 분리
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(
    california_housing.data, california_housing.target, test_size=0.2, random_state=42
)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(16512, 8) (16512,)
(4128, 8) (4128,)


In [6]:
# 표준 스케일링 전처리
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
from sklearn.linear_model import Ridge
# 모델 학습
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error

pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    #('lin', LinearRegression()),
    ('model', Ridge(alpha=100))
])

pipeline.fit(X_train, y_train)


# 회귀계수 확인 : alpha값에 따라 회귀계수 제곱의 합을 제어 (회귀계수 크기 관리)
model = pipeline.named_steps['model']
print('회귀계수 :', np.sum(model.coef_**2))

 # 모델 평가/예측
 # - R ^2, MSE, MAE, RSME
print('학습셋 R^2',  pipeline.score(X_train, y_train))
print('평가셋 R^2', pipeline.score(X_test, y_test))

print('학습셋 MES', mean_squared_error(y_train, pipeline.predict(X_train)))
print('평가셋 MES', mean_squared_error(y_test, pipeline.predict(X_test)))

print('학습셋 MAE', mean_absolute_error(y_train, pipeline.predict(X_train)))
print('평가셋 MAE', mean_absolute_error(y_test, pipeline.predict(X_test)))

print('학습셋 MES', root_mean_squared_error(y_train, pipeline.predict(X_train)))
print('평가셋 MES', root_mean_squared_error(y_test, pipeline.predict(X_test)))

회귀계수 : 5.28678801861077
학습셋 R^2 0.6764344307244856
평가셋 R^2 0.6585764543911401
학습셋 MES 0.4325354030927502
평가셋 MES 0.44740447954869134
학습셋 MAE 0.4735725589626248
평가셋 MAE 0.4773667977094845
학습셋 MES 0.6576742378204807
평가셋 MES 0.6688830088652957


## 최적의 alpha값 찾기

In [8]:
from sklearn.model_selection import cross_val_score

alphas = [0, 0.1, 1, 10, 100]

for alpha in alphas:
    pipeline = Pipeline([
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('model', Ridge(alpha=alpha))
    ])


    scores = -1 * cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    print(f'alpha {alpha}: MSE = {scores.mean():.4f}')

alpha 0: MSE = 10.4483
alpha 0.1: MSE = 10.0897
alpha 1: MSE = 7.5929
alpha 10: MSE = 1.7484
alpha 100: MSE = 0.4864


## L1
- L1 규제 방식을 구현한 Lasso 클래스를 사용할 수 있다.
- alpha값을 통해 특정 회귀계수를 0까지 제한, 특정 속성을 회귀계산에서 배제하는 것도 가능
- 특성을 그냥꺼버린다.

In [9]:
from sklearn.linear_model import Lasso

alphas = [0.003, 0.07, 0.1, 0.5, 1]

coef_df = pd.DataFrame()

for alpha in alphas:
    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)

    coef_df[f'alpha{alpha}'] = pd.Series(model.coef_,index=california_housing.feature_names)

    print(f'alpha = {alpha} : MSE = {mean_squared_error(y_train, model.predict(X_train))}')
coef_df

alpha = 0.003 : MSE = 0.5184280785246578
alpha = 0.07 : MSE = 0.6558951403937149
alpha = 0.1 : MSE = 0.6718399298159432
alpha = 0.5 : MSE = 0.9491447170182818
alpha = 1 : MSE = 1.3367782117894271


,alpha0.003,alpha0.07,alpha0.1,alpha0.5,alpha1
MedInc,0.838509,0.742618,0.710598,0.29852,0.0
HouseAge,0.124458,0.140497,0.106453,0.00000,0.0
AveRooms,-0.254950,-0.000000,-0.000000,0.00000,0.0
AveBedrms,0.299488,0.000000,0.000000,-0.00000,-0.0
Population,-0.000000,0.000000,-0.000000,-0.00000,-0.0
AveOccup,-0.037893,-0.000000,-0.000000,-0.00000,-0.0
Latitude,-0.864221,-0.039197,-0.011469,-0.00000,-0.0
Longitude,-0.835076,-0.000000,-0.000000,-0.00000,-0.0


## L1 + L2
L1규제, L2규제를 적절한 비율로 모두 적용하는 ElasticNet 선형회귀모델을 사용할 수 있다.


ElasticNet은 **회귀 분석** 기법 중 하나로, **Lasso**와 **Ridge**의 규제를 결합한 모델이다.

Lasso는 특성 선택에 효과적이고, Ridge는 모든 특성을 다루면서 모델을 규제한다.

ElasticNet은 이 두 가지 규제(L1과 L2)를 적절히 혼합하여 사용하는 방법이다.

**alpha 파라미터**
alpha는 a + b를 의미한다.
- a는 L1규제용 alpha값이다.
- b는 L2규제용 alpha값이다.

**l1_ratio 파라미터**
L1규제용 alpha값의 비율이다. $\frac{a}{a + b}$
- alpha가 10이고, l1_ratio가 0.7이면 a = 7, b = 3이다.
- alpha가 10이고, l1_ratio가 1이면 a = 10, b = 0이다. 즉, L1규제만 사용한다.
- alpha가 10이고, l1_ratio가 0이면 a = 0, b = 10이다. 즉, L2규제만 사용한다.


**수식:**
$$J(β) = RSS + α [ λ * ||β||₁ + (1 - λ) * ||β||₂² ]$$

- **RSS**: Residual Sum of Squares (예측 오차)
- **α**: 전체 규제 강도 (크면 규제가 강해짐)
- **λ**: L1과 L2 규제의 비율 조절 (0 ≤ λ ≤ 1)
  - λ = 1 → Lasso만 적용
  - λ = 0 → Ridge만 적용
- **||β||₁**: L1 노름 (∑|βᵢ|), 특성 선택
- **||β||₂²**: L2 노름 제곱 (∑βᵢ²), 계수 축소


**특징:**
- **Lasso와 Ridge의 장점을 결합**: ElasticNet은 Lasso의 **특성 선택** 능력과 Ridge의 **강한 규제** 특성을 모두 반영한다.
- **고차원 데이터에 적합**: 상관관계가 높은 특성이 많은 데이터나 차원이 높은 데이터에 적합하다.
- **Overfitting 방지**: 두 가지 규제를 혼합하여 과적합을 효과적으로 방지할 수 있다.

In [10]:
from sklearn.linear_model import ElasticNet

model = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('elasticnet', ElasticNet(alpha=0.1, l1_ratio=0.1, max_iter=5000)),
])

# 학습
model.fit(X_train, y_train)

# 평가
print('학습셋: ', model.score(X_train, y_train))
print('평가셋: ', model.score(X_test, y_test))

학습셋:  0.5767067319246053
평가셋:  0.5490213957896161


C:\Users\TJ\anaconda3\envs\ai_env2\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.352e+00, tolerance: 2.207e+00
  model = cd_fast.enet_coordinate_descent(


In [11]:
coef = model.named_steps['elasticnet'].coef_
pd.Series(coef)


0     0.736058
1     0.143314
2    -0.056360
3     0.058125
4     0.000000
5    -0.048079
6    -0.338281
7    -0.317353
8    -0.022980
9     0.041135
10    0.043769
11   -0.032512
12    0.004668
13   -0.000000
14    0.000000
15   -0.000000
16    0.036072
17   -0.000000
18   -0.000000
19    0.006235
20   -0.000000
21    0.000000
22   -0.011279
23    0.014046
24   -0.013380
25   -0.000000
26    0.011009
27    0.002934
28   -0.000000
29    0.000862
30   -0.000000
31    0.000000
32    0.000000
33   -0.000000
34    0.000432
35    0.000000
36    0.014095
37   -0.000000
38    0.000277
39    0.000000
40   -0.000000
41   -0.054121
42    0.000000
43    0.002321
dtype: float64

## 다중공선성 MultiCollinearity

- 특성간의 상관관계가 너무 높은 경우를 가리킨다.
- 주택데이터에서 면적 특성과 방의 크기 특성은 높은 상관관계(상관계수 0.8 이상)를 가질 수 있다.

다중공선성특성에 대한 회귀계수가 크게 학습이 되고, 이는 특정데이터에 민감산 과대적합을 유발한다.

==> 고추장, 고추가루, 불닭 소스 -> 상관관계가 높다.
근데 이제 요리할때 뭐를 얼마나 넣을지 정확히 결정을 못해서 모델 예측이 불안정해진다.

### 해결책
- 다중공선성 특성 제거
- 규제 모델을 사용한 회귀계수 억제



In [16]:
np.random.seed(42)

# 샘플데이터 생성
X = np.random.rand(100, 1)
X = np.column_stack((X, X ** 2 + 3, X ** 3 + 4))
y = 3 * np.random.randn(100,1) + np.random.randn(100,1)

# 컬럼간 상관계수 확인
# - df.corr()
# - np.corroef()


corr_mat = np.corrcoef(X, rowvar=False) # 행을 특성으로 지정 (기본값 True)
corr_mat

array([[1.        , 0.96876011, 0.91658615],
       [0.96876011, 1.        , 0.98569849],
       [0.91658615, 0.98569849, 1.        ]])

In [18]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 모델 생성
models = [
    LinearRegression(),
    Ridge(alpha=10),
    Lasso(alpha=0.1),
    ElasticNet(alpha=1, l1_ratio=0.2),

]

for model in models:
    model.fit(X_train, y_train)

    print(model.__class__.__name__)
    print('학습셋 MSE :', mean_squared_error(y_train, model.predict(X_train)))
    print('평가셋 MSE :', mean_squared_error(y_test, model.predict(X_test)))
    print('회귀계수 :', model.coef_)

LinearRegression
학습셋 MSE : 7.217015970597714
평가셋 MSE : 10.176684644740662
회귀계수 : [[-17.43071311  26.7371064  -11.43186193]]
Ridge
학습셋 MSE : 7.818478096471177
평가셋 MSE : 11.046822732679251
회귀계수 : [-0.52065163 -0.08262629  0.15334546]
Lasso
학습셋 MSE : 7.900941066505652
평가셋 MSE : 11.193791774410979
회귀계수 : [-0.05683636 -0.         -0.        ]
ElasticNet
학습셋 MSE : 7.9125823291145805
평가셋 MSE : 11.213652084293873
회귀계수 : [-0. -0. -0.]


In [ ]:
()